In [1]:
import pandas as pd

### Calling review data

In [2]:
df = pd.read_csv('data/cafe_reviews.csv')
df.head()

,store,rating,description,reviews
0,오드랑베이커리,4.818182,NaN,"['마농바게트 맛있어요~~', '맛있어요.', '까르보나라 빵 진짜맛있어요', '마..."
1,Bake things better,4.875000,안녕하세요! 베이크 띵스 베러입니다.\n우리 매장은 테이크 아웃을 전문으로 한 베이...,['쿠키만 사려 했는데 스콘이 너무 맛있어보여서… 대체적으로 다 맛있었어요~~ 말차...
2,파파무베이커리,4.656863,아빠가 딸을위해 만드는 건강한빵으로 아이가 아빠를부르는 옹알이의 파파무입니다^^천연...,"['사장님도 친절하고 빵도 맛있어요!', '빵도 케익도 커피도 맛있고 사장님 인성..."
3,텍스처베이커리,4.611111,텍스처 베이커리는 가정식 디저트를 추구합니다.\n꾸며지지 않은 러프한 디저트를 즐겨...,"['키쉬종류가 많아서 좋아요', '매장인테리어 귀엽구 빵도 일반적이지 않은 빵들이 ..."
4,가야베이커리,5.000000,NaN,['규모는 작지만 빵이 맛있고 가격도 저렴해서 자주 방문하고 있습니다. 사장님 친절...


In [3]:
df['reviews']

0      ['마농바게트 맛있어요~~', '맛있어요.', '까르보나라 빵 진짜맛있어요', '마...
1      ['쿠키만 사려 했는데 스콘이 너무 맛있어보여서… 대체적으로 다 맛있었어요~~ 말차...
2      ['사장님도 친절하고 빵도 맛있어요!', '빵도 케익도 커피도 맛있고  사장님 인성...
3      ['키쉬종류가 많아서 좋아요', '매장인테리어 귀엽구 빵도 일반적이지 않은 빵들이 ...
4      ['규모는 작지만 빵이 맛있고 가격도 저렴해서 자주 방문하고 있습니다. 사장님 친절...
                             ...                        
929    ['맛집', '쑥빵 맛있어요', '평범한 동네 빵집이었어요', '', '굿', '굿...
930    ['사장님 너무너무 친절하시규 포크도 챙겨주셔서 감동..ㅠㅠ 크레이크 케이크는 첨인...
931    ['가게는 아담하지만 핫도그 정말 맛있어요.', '맛나요~~', '평범하지만 맛있어...
932                                                   []
933    ['부담없이 편한곳~~추천추천', '전체적으로 너무 깔끔하고 넓어서 너무좋아요 무엇...
Name: reviews, Length: 934, dtype: object

### Preprocess each reviews